In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import scienceplots

import os
import sys
from dotenv import load_dotenv

load_dotenv()
REPO_PATH = os.getenv("REPO_PATH")

# Import main utility functions
sys.path.insert(0, rf'{REPO_PATH}src')
plt.style.use('science')

In [ ]:
FUTURES = ['CLc1', 'LCOc1']
TOPICS = ['CRU', 'CWP', 'CEN']

dfs = {
    future: pd.read_csv(
        os.path.join(
            REPO_PATH,
            'data',
            'prepared_data',
            f"{future}_5min_resampled.csv"
        ),
        index_col='date',
        parse_dates=True
    ) for future in FUTURES
}

df = dfs['CLc1']

topic_names: dict[int, str] = {
    0: 'Securities and Commodity Markets',
    1: 'Interest Rates and Economic Policy',
    2: 'Geopolitical Conflicts',
    3: 'Banking and Finance',
    4: 'Oil and Gas Production'
}


In [ ]:
CT_VADER = [col for col in df.columns if 'SI' in col and 'CT' in col and 'VADER' in col]


fig, ax = plt.subplots(1, 1, figsize=(10, 5), dpi=200)
ax2 = ax.twinx()

top_num = [string.split('_')[1] for string in CT_VADER]
labels = [topic_names[int(num)] for num in top_num]

df[CT_VADER].rolling(288*3).mean().plot(ax=ax, lw=0.4)
# add legend
ax.legend(labels)


In [ ]:
CT_textblob = [col for col in df.columns if 'SI' in col and 'CT' in col and 'TextBlob' in col]

fig, ax = plt.subplots(1, 1, figsize=(10, 5), dpi=200)
ax2 = ax.twinx()

top_num = [string.split('_')[1] for string in CT_textblob]
labels = [topic_names[int(num)] for num in top_num]

df[CT_textblob].rolling(288*3).mean().plot(ax=ax, lw=0.4)
# add legend
ax.legend(labels)